##Propagation through beamline with several optic elements

This example uses wave optics software based on SRW core library <https://github.com/ochubar/SRW>, available through WPG interactive framework <https://github.com/samoylv/WPG> 

#TODO
more short explaination about example

![Image of simulated beamline](diagramm.png)






In [ ]:
%pylab inline

####Import modules and necessary core and helper functions

In [ ]:
import os
import sys
#sys.path.insert(0,'/home/meyerann/WPG')
sys.path.insert(0,os.path.join('..','..'))

import time
import copy
import numpy
import pylab

#Base wavefront class
from wpg import Wavefront

#import SRW helpers functions
from wpg.useful_code.srwutils import AuxTransmAddSurfHeightProfileScaled

from wpg.srwlib import srwl, SRWLOptD, SRWLOptL, SRWLOptC, SRWLOptA, SRWLOptMirEl, SRWLOptT
#Gaussian beam generator
from wpg.generators import build_gauss_wavefront_xy
from wpg.beamline import Beamline
from wpg import optical_elements  # wrapers for optical elements
 
#helper functions
#from wfrutilsAKM import plot_wfront
from wpg.useful_code.wfrutils import plot_wfront, get_mesh, print_beamline, calculate_fwhm_x, calculate_fwhm_y, plot_1d, plot_2d

pylab.ion()

In [ ]:
def calculate_theta_fwhm_cdr(ekev,qnC):
    """
    Calculate angular divergence using formula from XFEL CDR2011
    
    :param ekev: Energy in keV
    :param qnC: e-bunch charge, [nC]
    :return: theta_fwhm [units?]
    """
    theta_fwhm = (17.2 - 6.4 * sqrt(qnC))*1e-6/ekev**0.85
    return theta_fwhm

In [ ]:
def defineOPD(opTrErMirr, mdatafile, ncol, delim, Orient, theta, scale):
    """
    Define optical path difference (OPD) from mirror profile, i.e. ill the struct opTrErMirr
    
    :params mdatafile: an ascii file with mirror profile data
    :params ncol: number of columns in the file
    :params delim: delimiter between numbers in an row, can be space (' '), tab '\t', etc
    :params orient: mirror orientation, 'x' (horizontal) or 'y' (vertical)
    :params theta: incidence angle
    :params scale: scaling factor for the mirror profile    
    """
    heightProfData = numpy.loadtxt(mdatafile).T
    AuxTransmAddSurfHeightProfileScaled(opTrErMirr, heightProfData, Orient, theta, scale)
    pylab.figure()
    plot_1d(heightProfData,'profile from ' + mdatafile,'x (m)', 'h (m)')

In [ ]:
def defineEFM(orient,p,q,thetaEFM,theta0,lengthEFM):
    """
    A wrapper to a SRWL function SRWLOptMirEl() for defining a plane elliptical focusing mirror propagator
    
    :param Orient:    mirror orientation, 'x' (horizontal) or 'y' (vertical)
    :param p:  the distance to two ellipsis centers
    :param q:  the distance to two ellipsis centers
    :param thetaEFM:  the design incidence angle in the center of the mirror
    :param theta0:    the "real" incidence angle in the center of the mirror
    :param lengthEFM: mirror length, [m]
    :return: the struct opEFM
    """
    if orient == 'x':     #horizontal plane ellipsoidal mirror
        opEFM = SRWLOptMirEl(_p=p, _q=q, _ang_graz=thetaEFM, _r_sag=1.e+40, _size_tang=lengthEFM, 
            _nvx=cos(theta0), _nvy=0, _nvz=-sin(theta0), _tvx=-sin(theta0), _tvy=0, _x=0, _y=0, _treat_in_out=1) 
    elif orient == 'y': #vertical plane ellipsoidal mirror
        opEFM = SRWLOptMirEl(_p=p, _q=q, _ang_graz=thetaEFM, _r_sag=1.e+40, _size_tang=lengthEFM, 
            _nvx=0, _nvy=cos(theta0), _nvz=-sin(theta0), _tvx=0, _tvy=-sin(theta0), _x=0, _y=0, _treat_in_out=1)
    else:
        raise TypeError('orient should be "x" or "y"')
    return opEFM

In [ ]:
def defineBladeBL(width,zCenter,zEnd,orient='x',kFresnel=1,sizeX=5e-2,sizeY=5e-2):
    """
    A wrapper to a SRWL function SRWLOptA() for defining a beamline with a slit with 2 rectangular obstacles and 
    calculates expected position of k. minima of interference fringe caused by 1. and 2. obstacle.
    
    :param width:  slit width [m]
    :param zCenter:  position of center of blades [m]
    :param zEnd:  observation point position [m]
    :param orient: direction of shifting the slit center, 'x' (horizontal = default) or 'y' (vertical)
    :param kFresnel:  number of minimum on Fresnel diffraction curve, which position will be printed out
    :param sizeX: horizontal size of blade [m]
    :param sizeY: vertical size of blade [m]
    :return: the struct BL and parameter distanceBetweenObstacles
    """
    distBetweenObstacles = 0.30
    Position2Obstacle = zCenter+distBetweenObstacles/2
    distDriftAfter1Obstacle = zEnd - Position2Obstacle + distBetweenObstacles
    distDriftAfter2Obstacle = zEnd - Position2Obstacle

    F = width**2/(distDriftAfter1Obstacle*lamda)
    print 'Fresnel number after 1. obstacle in distance %.3f m is %.3f' %(distDriftAfter1Obstacle, F)
    x_n = sqrt((distDriftAfter1Obstacle)*lamda*(2*kFresnel-1/4))
    print 'expected %f minima of interference fringe after 1. obstacle at: %.9f m' %(kFresnel,x_n)

    F = width**2/(distDriftAfter2Obstacle*lamda)
    print 'Fresnel number after 2. obstacle in distance %.3f m is %.3f' %(distDriftAfter2Obstacle, F)
    x_n = sqrt((distDriftAfter2Obstacle/2)*lamda*(2*kFresnel-1/4))
    print 'expected %f minima of interference fringe after 2. obstacle at: %.9f m' %(kFresnel,x_n)

    F = width**2/(distBetweenObstacles*lamda)
    print 'Fresnel number after both obstacle in distance %.3f m is %.3f' %(distDriftAfter2Obstacle, F)

    L0 = Beamline()
    L1 = Beamline()
    L2 = Beamline()
    
    DriftOp2Op = SRWLOptD(distBetweenObstacles)
    DriftAfter2Obstacle = SRWLOptD(distDriftAfter2Obstacle)
    
    if orient == 'x':
        opOpstacle1 = SRWLOptA('r', 'o',sizeX,sizeY,   (sizeX/2 + width/2),0)
        opOpstacle2 = SRWLOptA('r', 'o',sizeX,sizeY,  -(sizeX/2 + width/2),0)
        L0.append(opOpstacle1,    optical_elements.Use_PP(zoom_h=1.0, sampling_h=1.0/0.25, semi_analytical_treatment=0))
        L1.append(DriftOp2Op,     optical_elements.Use_PP(semi_analytical_treatment=0))
        L1.append(opOpstacle2,    optical_elements.Use_PP(zoom_h=1/2.5, sampling_h=1.0,  semi_analytical_treatment=0))
        L2.append(DriftAfter2Obstacle, optical_elements.Use_PP(zoom_h=2.4, sampling_h=1.0/0.4,semi_analytical_treatment=1)) 
    elif orient == 'y':
        opOpstacle1 = SRWLOptA('r', 'o',sizeX,sizeY,0, (sizeY/2 + width/2))
        opOpstacle2 = SRWLOptA('r', 'o',sizeX,sizeY,0,-(sizeY/2 + width/2))
        L0.append(opOpstacle1,    optical_elements.Use_PP(zoom_v=1.0, sampling_v=1.0/0.25, semi_analytical_treatment=0))
        L1.append(DriftOp2Op,     optical_elements.Use_PP(semi_analytical_treatment=0))
        L1.append(opOpstacle2,    optical_elements.Use_PP(zoom_v=1/2.5, sampling_v=1.0,  semi_analytical_treatment=0))
        L2.append(DriftAfter2Obstacle, optical_elements.Use_PP(zoom_v=2.4, sampling_v=1.0/0.4,semi_analytical_treatment=1))    
    else:
        raise TypeError('orient should be "x" or "y"')
    

    
    return L0,L1,L2, distBetweenObstacles

###  Gaussian wavefront parameters

Before building initial gaussian wavefront the beam parameters, which defines the behavior and geometry of a Gaussian beam, must be calculated. 

wave length of beam: $\lambda=\frac{12.4}{E[kev]}$ 

angular divergence: $\theta_{FWHM}$ is calculated by using the empirical formula from XFEL CDR2011

beam waist: $\omega_0 = \frac{\lambda}{\pi\theta}$

Rayleigh range: ${z}_R=\frac{\pi{\omega_0}^2}{\lambda}$  At this point the beam is widened to $\sqrt{2}\omega_0$

FWHM at ${z}_R$: $FWHM_{z_R} = \theta_{FWHM}{z}_R$ --> weil sin(theta[theta im bogenmass])=x\laenge=theta

FWHM of amplitude: $\frac{1}{2} = exp(-\frac{x^2}{w^2})$ --> $x = w\sqrt{-ln\frac{1}{2}}$ --> $x = 2\sigma\sqrt{ln2}$

$\sigma$ of amplitude: $ \sigma_{amp} = \frac{FWHM}{2\sqrt{ln2}} $

Afterwards the parameters will be printed out, so the expected FWHM can be compared with the propagated.

In [ ]:
#wavefront parameters
qnC = 0.1  # e-bunch charge, [nC]
pulse_duration = 9.e-15 # [s]      
ekev = 0.8  # Energy in [keV]
#ekev = 0.4
#ekev = 1.6

lamda = 12.4*1e-10/ekev # wavelength [AKM]
theta_fwhm = calculate_theta_fwhm_cdr(ekev,qnC)#angular divergence
w0 = lamda/(pi*theta_fwhm) # beam waist
z_R = (pi*w0**2)/lamda #Rayleigh range
fwhmAtZ_R = theta_fwhm*z_R #FWHM at Rayleigh range
sigmaAmp = w0/(2*sqrt(log(2))) #sigma of amplitude

#Print out the calculated wavefront parameters and expected FWHM
print 'theta_fwhm: %.2f urad' % (theta_fwhm*1e6)
print 'w0 %.3f um:' %(w0*1e6)
print 'lamda %.3f nm:' %(lamda*1e9)
print '2*z_R: %.3f m:' %(2*z_R)
print 'sigma_Amplitude %.3f um:' %(sigmaAmp*1e6)

### initial Gaussian wavefront
With the calculated beam parameters the initial wavefront is build with 400x400 data points and in distance of the first vertical elliptical focusing mirror at 300m. For further propagation the built wavefront should be stored. After plotting the wavefront the FWHM should be printed out for comparing.

Expected FWHM at first screen or focusing mirror: $FWHM_{d} = \theta_{FWHM}d$

####gaussian beam radius and size at some point 
$\omega(z) = \omega_0*\sqrt{1+(\frac{z\lambda}{\pi\omega_0^2})^2}$ 

$\frac{1}{z_R} = \frac{\lambda}{\pi\omega_0^2}$

$\omega(z) = \omega_0*\sqrt{1+(\frac{z}{z_R})^2}$

####beam size is a little bit (ca 200 nm )smaller than expected. Is it because the calculating of the expected FWHM is approximatly???

beam radius at several point
$\omega(z) = \omega_0*\sqrt{1+(\frac{z\lambda}{\pi\omega_0^2})^2}$ 

$\frac{1}{z_R} = \frac{\lambda}{\pi\omega_0^2}$

$\omega(z) = \omega_0*\sqrt{1+(\frac{z}{z_R})^2}$



In [ ]:

HEFM = 284.0 # [m] first horizontal focussing mirror M2
VLens = 300.0 # [m] first vertical focussing mirror
print 'expected FWHM at distance %.1f: %.2f mm' %(HEFM,theta_fwhm*HEFM*1e3)

#quintuple beam radius at horizontal lens distance to get the range of the wavefront 
range_xy = w0*sqrt(1+(VLens/z_R)**2) *5
print 'range_xy :', range_xy

#number of points
np = 400

#build wavefront
wfr = build_gauss_wavefront_xy(nx=np, ny=np, ekev=ekev, xMin=-range_xy/2, xMax=range_xy/2, 
                               yMin=-range_xy/2, yMax=range_xy/2, sigX=sigmaAmp, 
                               sigY=sigmaAmp,d2waist=HEFM)
#init WPG Wavefront with helper class
mwf = Wavefront(wfr)

#defining name HDF5 file for storing wavefront
strOutInDataFolder = 'data_common'
wfrName = os.path.join(strOutInDataFolder,'gwf'+'.h5')
#store wavefront to HDF5 file 
#mwf.store_hdf5(wfrName)
print 'saving WF to %s' %wfrName

#plot the wavefront
plot_wfront(mwf, title_fig='before M2 at '+str(HEFM)+' m', isHlog=False, isVlog=False,
            i_x_min=1e-6, i_y_min=1e-6, orient='y', onePlot=True, bPlotPha=False)
print ' '
print ' FWHM at distance %.1f: %.3f mm' %(HEFM,calculate_fwhm_x(mwf)*1e3)

###Horizontal elliptical focusing offset mirror:

The wavefront will be focused with a focusing offset mirror (M2) at distance (284.0m) in horizontal plane. Before focusing the beam the expected beam size after the propagation at point of focus and calculating distances of focus.

Focal length for thin lenses will be calculated by

$\frac{1}{f}=\frac{1}{p}+\frac{1}{q}$

The expected FWHM at focus will be calculated with proportion of the distances after and before the lens multiplied by the FWHM at source 

$FWHM_{focus}=\frac{q}{p}*FWHM_{source}$

In [ ]:
horFocus = 374.0 #given point of where the focus should be
q = VLens-HEFM #distance between horizontal focusing lens and given point of focus

### Building beamline with horizontal focusing mirror
The beamline consists of an aperture infront of the horizontal elliptical focussing mirror, to clean up the beam, the focussing mirror and the drift after lens to the vertical focussing lens. So the wavefront is propagated from first horizontal lens at 284m to vertical lens at 300m.

The size of the horizontal FWHM at horizontal focus varies according to the change of the angle of the offset focussing mirror. The beam gets a smaller focus with bigger angle. 

The length of the mirror is realized with the aperture.
The size of the aperture is calculated with angle and length of mirror $l_M$
$sin\theta = \frac{x}{l_M}$ --> $x = sin\theta*l_M$ with $sin\theta = \theta$ for small angles in urad.


In [ ]:
thetaOM = 6e-3 
horApM1 = 0.8*thetaOM
opApHEFM =  SRWLOptA('r', 'a',horApM1,range_xy) 
opApGen=SRWLOptA('r', 'a',range_xy,range_xy)

thetaEM = 52.4e-3   # incidence angle of elliptical  mirror, 3 deg
lengthM = 0.5 #[m] Mirror length

dHEFM2VL = VLens-HEFM #distance between horizontal focusing lens and given point of focus

HorizontalEFM = defineEFM(orient='x',p=HEFM ,q=dHEFM2VL,thetaEFM=thetaEM,theta0=thetaEM,
                          lengthEFM=lengthM) # define horizontal elliptical focussing mirror
DriftHEFM_VLens = SRWLOptD(dHEFM2VL) #define the drift between

BL0 = Beamline()
BL0.append(opApHEFM, optical_elements.Use_PP(sampling_h=1.0/0.125, semi_analytical_treatment=0))
BL0.append(HorizontalEFM, optical_elements.Use_PP(semi_analytical_treatment=1))
BL0.append(DriftHEFM_VLens,optical_elements.Use_PP(semi_analytical_treatment=1))
BL0.append(opApGen,optical_elements.Use_PP(zoom_h=1/10., semi_analytical_treatment=0))
BL0.append(opApHEFM, optical_elements.Use_PP(sampling_v=1.0/0.5, semi_analytical_treatment=0))
print BL0


# reload the wavefront (good practise while debugging beamline)
mwf = Wavefront()
#mwf.load_hdf5(wfrName)
print 'loading WF from %s' %wfrName

BL0.propagate(mwf)

print 'Plot mit obstacle'
plot_wfront(mwf, title_fig='before M3 at '+str(VLens)+' m', isHlog=False, isVlog=False, 
            i_x_min=1e-4, i_y_min=1e-4, orient='x', onePlot=True, bPlotPha=False)
print ' '
wfr0Name = os.path.join(strOutInDataFolder,'wBL0vh'+'.h5')
#store wavefront to HDF5 file 
#mwf.store_hdf5(wfr0Name)



### Building beamline with focusing mirror
The beamline consists of an aperture infront of the horizontal elliptical focussing mirror, to clean up the beam, the focussing mirror and the drift after lens to the vertical focussing lens. So the wavefront is propagated from first horizontal lens at 284m to vertical lens at 300m.

The size of the horizontal FWHM at horizontal focus varies according to the change of the angle of the offset focussing mirror. The beam gets a smaller focus with bigger angle. 

The length of the mirror is realized with the aperture.
The size of the aperture is calculated with angle and length of mirror $l_M$
$sin\theta = \frac{x}{l_M}$ --> $x = sin\theta*l_M$ with $sin\theta = \theta$ for small angles in urad.

### Building beamline with vertical focusing mirror
The beamline consists of the vertical focussing lens at 300m and it drifts through the horizontal focus at 374m to the vertical focus at 400m. Two blades are placed around the horizontal focus to clean up the beam.

The size of the the horizontal FWHM varies according to the change of the slit size, which is created by the blades.  

In [ ]:
vertFocus = 400
qN = vertFocus - VLens

slit_size = 200e-6
BL2 = Beamline()
BL3 = Beamline()
BL4 = Beamline()
#define beamline with 1. blade on focus
BL2,BL3,BL4, distBetweenObstacles = defineBladeBL(width=slit_size, zCenter=horFocus+0.15, zEnd=vertFocus, orient='x', kFresnel=2)
DriftHM2Op = SRWLOptD(q)

VertLens = SRWLOptL(1e23,qN)
DriftVLens_hFocus = SRWLOptD(horFocus-VLens-distBetweenObstacles)

BL01 = Beamline()

BL01.append(VertLens, optical_elements.Use_PP(semi_analytical_treatment=1))
BL01.append(DriftVLens_hFocus,optical_elements.Use_PP(semi_analytical_treatment=1))
BL01.append(opApGen,optical_elements.Use_PP(zoom_h=1/1.25,zoom_v=1/1.6))

BL01.propagate(mwf)
BL2.propagate(mwf)
BL3.propagate(mwf)
BL4.propagate(mwf)

print 'Plot mit obstacle'
plot_wfront(mwf, title_fig='at VF at '+str(vertFocus)+' m', isHlog=False, isVlog=False, 
            i_x_min=1e-4, i_y_min=1e-4, orient='x', onePlot=True, bPlotPha=False)
print ' '


### Building beamline through vertical focus

The beamline consists of two blades to clean up the beam. The first one is at point of focus and the second is in opposite site of the beam behind the focus. After the second blade the beam propagates to the screen at 402m.

In [ ]:

slit_size = 200e-6
BL5 = Beamline()
BL6 = Beamline()
BL7 = Beamline()
#define beamline with 1. blade on focus
BL5,BL6,BL7, distBetweenObstacles = defineBladeBL(width=slit_size, zCenter=vertFocus+0.15, zEnd=vertFocus+20, orient='y', kFresnel=2)
BL7.append(opApGen,optical_elements.Use_PP(zoom_h=1/9.,zoom_v=1/9.))

BL5.propagate(mwf)
BL6.propagate(mwf)
BL7.propagate(mwf)

print 'Plot mit obstacle'
plot_wfront(mwf, title_fig='at KB aperture at '+str(vertFocus+2)+' m', isHlog=False, isVlog=False, 
            i_x_min=1e-4, i_y_min=1e-4, orient='x', onePlot=True, bPlotPha=False)
print ' '